In [1]:
import math
import pandas as pd


def distance(word1, word2):
    d = 0
    for i in range(len(word1)):
        if word1[i] != word2[i]:
            d += 1
    return d

In [6]:
# 	F(А) = {(а1; 000000); (а2; 001001); (а3; 001110); (а4; 011011); (а5; 111001); (а6; 111111)}.
codewords = ["000000", "001001", "001110", "011011", "111001", "111111"]
codewords_list = list(codewords)
pe = 1e-1
prob_not_detecting_error_ai = []
joint_matrix = []
for i, code_a in enumerate(codewords_list):
    joint_matrix.append([])
    for j, code_b in enumerate(codewords_list):
        d = distance(code_a, code_b)
        n = len(code_b)
        val = math.pow(pe, d) * math.pow(1 - pe, n - d)
        joint_matrix[i].append(val)
for i in range(len(codewords_list)):
    sum = 0
    for j in range(len(codewords_list)):
        sum += joint_matrix[j][i]
    joint_matrix[i][i] = 1 - sum + joint_matrix[i][i]
    prob_not_detecting_error_ai.append(1 - joint_matrix[i][i])
pd.DataFrame(joint_matrix)

,0,1,2,3,4,5
0,0.992547,0.006561,0.000729,0.000081,0.000081,0.000001
1,0.006561,0.979507,0.000729,0.006561,0.006561,0.000081
2,0.000729,0.000729,0.997075,0.000729,0.000009,0.000729
3,0.000081,0.006561,0.000729,0.979507,0.006561,0.006561
4,0.000081,0.006561,0.000009,0.006561,0.980227,0.006561
5,0.000001,0.000081,0.000729,0.006561,0.006561,0.986067


In [7]:
prob_not_detecting_error_ai_df = pd.DataFrame(prob_not_detecting_error_ai)
prob_not_detecting_error_ai_df

[0.007453000000000043, 0.020493000000000094, 0.0029249999999999554, 0.020493000000000094, 0.01977300000000004, 0.013932999999999973]


,0
0,0.007453
1,0.020493
2,0.002925
3,0.020493
4,0.019773
5,0.013933


In [3]:
from copy import deepcopy

probabilities_list = [0.15, 0.1, 0.05, 0.3, 0.25, 0.15]
channel_matrix = deepcopy(joint_matrix)
for i in range(len(joint_matrix)):
    for j in range(len(joint_matrix)):
        channel_matrix[i][j] = joint_matrix[i][j] * probabilities_list[i]
channel_matrix_df = pd.DataFrame(channel_matrix)
channel_matrix_df

,0,1,2,3,4,5
0,1.488820e-01,0.000984,0.000109,0.000012,1.215000e-05,1.500000e-07
1,6.561000e-04,0.097951,0.000073,0.000656,6.561000e-04,8.100000e-06
2,3.645000e-05,0.000036,0.049854,0.000036,4.500000e-07,3.645000e-05
3,2.430000e-05,0.001968,0.000219,0.293852,1.968300e-03,1.968300e-03
4,2.025000e-05,0.001640,0.000002,0.001640,2.450567e-01,1.640250e-03
5,1.500000e-07,0.000012,0.000109,0.000984,9.841500e-04,1.479100e-01


In [4]:
column_sums = channel_matrix_df.sum()

# Create a new DataFrame with a single row containing the column sums
sums_df = pd.DataFrame([column_sums.values], columns=column_sums.index, index=["$p(b_j)$"])
sums_df


,0,1,2,3,4,5
$p(b_j)$,0.149619,0.102592,0.050366,0.297181,0.248678,0.151563


In [5]:
channel_matrix_p_ai_bj = deepcopy(channel_matrix)
for i, row in enumerate(channel_matrix_p_ai_bj):
    for j, col in enumerate(row):
        channel_matrix_p_ai_bj[i][j] = channel_matrix_p_ai_bj[i][j] / column_sums[j]
pd.DataFrame(channel_matrix_p_ai_bj)

,0,1,2,3,4,5
0,0.995072,0.009593,0.002171,0.000041,0.000049,9.896855e-07
1,0.004385,0.954760,0.001447,0.002208,0.002638,5.344302e-05
2,0.000244,0.000355,0.989824,0.000123,0.000002,2.404936e-04
3,0.000162,0.019186,0.004342,0.988798,0.007915,1.298665e-02
4,0.000135,0.015988,0.000045,0.005519,0.985438,1.082221e-02
5,0.000001,0.000118,0.002171,0.003312,0.003958,9.758962e-01
